# Chapter 1. The Python Data Model

Data model as a description of Python as a framework. If formalizes the interface of the building blocks of the language itself, such as sequence, iterators, function, classes, context managers, and so on.

While coding with any framwork, you spend a lot of time implementing methods that are called by framework. The same happens when you leverage the Python data model. The Python interpreter invokes special methods to perform basic object operations, often triggered by special syntax. The special method names are always written with leading and trailing double underscores (i.e. `__getitem__`). For exmaple, the syntax `obj[key]` is supportted by the `__getitem__` special method.

The special method names allow your object to implement, support, and iteract with basic language constructs such as:

* Iteration
* Collections
* Attribute access
* Operator overloading
* Function and method invocation
* Object creation and destruction
* String representation and formatting
* Managed contexts

## A Python Card Deck

The following is a every simple example, but it demonstrates the power of implementing just two methods, `__getitem__` and `__len__`.

In [24]:
import collections

In [25]:
Card = collections.namedtuple('Card', ['rank', 'suit'])

In [26]:
class FrenchDeck:
    ranks = [str(c) for c in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()
    
    
    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                                        for rank in self.ranks]

    def __len__(self):
        return len(self._cards)
    
    def __getitem__(self, position):
        return self._cards[position]

In [27]:
beer_card = Card('7', 'diamonds')

In [28]:
beer_card

Card(rank='7', suit='diamonds')

In [29]:
deck = FrenchDeck()

In [30]:
len(deck)

52

In [31]:
deck[0]

Card(rank='2', suit='spades')

In [32]:
deck[-1]

Card(rank='A', suit='hearts')

Should we create a method to pick a random card? No need. Python already has a function to get a random item from a sequence: `random.choice` We can just use it on a deck instance:

In [33]:
import random

In [34]:
random.choice(deck)

Card(rank='10', suit='clubs')

In [35]:
random.choice(deck)

Card(rank='4', suit='clubs')

We've just seen two advantage of using special methods to leverage the Python data model:

* The users of your classes don't have to memorize arbitrary method names fro standard operations
* It's easy to benefit from the rich Python standard library and avoid reinventing the wheel, like the `random.choice` function.

In [36]:
deck[:3]

[Card(rank='2', suit='spades'),
 Card(rank='3', suit='spades'),
 Card(rank='4', suit='spades')]

Interation if often implicit. If a collection has no `__contains__` method, the `in` operation does a sequential scan. Case in point: in works with our FrechDeck class because it is iterable.

In [37]:
Card('Q', 'hearts') in deck

True

In [38]:
Card('7', 'beats') in deck

False

How about sorting? A common system of ranking cards is by rank, then by suit in the order of spades, then hearts, diamonds, and clus.

In [39]:
suit_values = dict(spades=3, hearts=2, diamonds=1, clubs=0)

In [40]:
def spades_high(card):
    rank_value = FrenchDeck.ranks.index(card.rank)
    return rank_value * len(suit_values) + suit_values[card.suit]

In [41]:
for card in sorted(deck, key=spades_high):
    print(card)

Card(rank='2', suit='clubs')
Card(rank='2', suit='diamonds')
Card(rank='2', suit='hearts')
Card(rank='2', suit='spades')
Card(rank='3', suit='clubs')
Card(rank='3', suit='diamonds')
Card(rank='3', suit='hearts')
Card(rank='3', suit='spades')
Card(rank='4', suit='clubs')
Card(rank='4', suit='diamonds')
Card(rank='4', suit='hearts')
Card(rank='4', suit='spades')
Card(rank='5', suit='clubs')
Card(rank='5', suit='diamonds')
Card(rank='5', suit='hearts')
Card(rank='5', suit='spades')
Card(rank='6', suit='clubs')
Card(rank='6', suit='diamonds')
Card(rank='6', suit='hearts')
Card(rank='6', suit='spades')
Card(rank='7', suit='clubs')
Card(rank='7', suit='diamonds')
Card(rank='7', suit='hearts')
Card(rank='7', suit='spades')
Card(rank='8', suit='clubs')
Card(rank='8', suit='diamonds')
Card(rank='8', suit='hearts')
Card(rank='8', suit='spades')
Card(rank='9', suit='clubs')
Card(rank='9', suit='diamonds')
Card(rank='9', suit='hearts')
Card(rank='9', suit='spades')
Card(rank='10', suit='clubs')
Ca

> How About Shuffling? 
As implemented so far, a FrenchDeck cannot be shuffled, because it is immutable: the cards and their positions cannot be changed, except by violating encapsulation and handling the _cards attribute directly.

## How Special Methods Are Used

But for built-in types like `list`, `str`, `bytearray`, and so on, the interpreter taks a shortcut: the CPython implementation of `len()` actually returns the value of the `ob_size` field in the `PyVarObject` C struct that represents any variable-sized built-in object in memory. This much faster than callinga method.

`object.h`

```c
/* Nothing is actually declared to be a PyObject, but every pointer to
 * a Python object can be cast to a PyObject*.  This is inheritance built
 * by hand.  Similarly every pointer to a variable-size Python object can,
 * in addition, be cast to PyVarObject*.
 */
typedef struct _object {
    _PyObject_HEAD_EXTRA
    Py_ssize_t ob_refcnt;
    PyTypeObject *ob_type;
} PyObject;

/* Cast argument to PyObject* type. */
#define _PyObject_CAST(op) ((PyObject*)(op))
#define _PyObject_CAST_CONST(op) ((const PyObject*)(op))

typedef struct {
    PyObject ob_base;
    Py_ssize_t ob_size; /* Number of items in variable part */
} PyVarObject;
````

More often than not, the special method call is implicit. For example, the statement for i in x: actually causes the invocationof `iter(x)` which in turn may call `x.__iter__()` if that is aviable.

Mormally, your code should not have many direct calls to special mthods. Unless you are doing a lot of metaprogramming, you should be implementing special methods more often than invoking them explicity. The only special method that is frequently called by user code directly is `__init__`, to invoke the initializer of the superclass in your own `__init__` implementation.

If you need to invoke a special mthod, it is usually better to call the related built-in function(e.g. len, iter, str, etc). These built-in call the corresponding special method.

### Emulating Numerical Types

We will implement a class to represent two-dimensional vectors - that is Euclidean vectors like those used in math and physics.

In [42]:
import math

In [65]:
class Vecotr:
    
    def __init__(self, x=0, y=0):
        self.x = x
        self.y = y
        
    def __repr__(self):
        return 'Vector(%r %r)' % (self.x, self.y)
  
    def __add__(self, other):
        x = self.x + other.x
        y = self.y + other.y
        return Vecotr(x, y)

    def __abs__(self):
        return math.hypot(self.x, self.y)
    
    def __bool__(self):
        return bool(abs(self))
    
    def __mul__(self, scalar):
        return Vecotr(self.x*scalar, self.y*scalar)

In [67]:
v = Vecotr(3, 4)

In [68]:
print(v)

Vector(3 4)


In [56]:
v * 3

Vector(9 12)

In [57]:
v

Vector(3 4)

In [58]:
abs(v)

5.0

### String Representation

The `__repr__` special method is called by the `repr` built-in to get the representation of the object for inspection. If we did not implement `__repr__`, vector instance would be shown in the console like "<Vecotr at 0x106d5ed30>"

> Speaking of the % operator and the `str.format` method, you will notice I use both in this book, as does the Python community at large. I am increasing favoring the more powerful `str.format`.

If you only implement one the special methods, choose `__repr__`, because when no custom `__str__` is aviable, Python will call `__repr__` as a fallback.

### Boolean Value of a Custom Type

Although Python has a bool type, it accepts any object in a boolean contenxt.

By default, instances of user-defined classes are considered truthy, unless either `__bool__` or `__len__` is implemented. Basically, `bool(x)` call `x.__bool__()` and uses the result. If `__bool_` is not implemented, Python tries to invoke `x.__len__`, and if that return zeros, `bool` returns False. Otherwise bool retuns True.

A faster implementation of `Vector.__bool__` is this:

```py
def __bool__(self):
    return bool(self.x or self.y)
```

## Overview of Special Methods

[Data Model Python Reference](docs.python.org/3/reference/datamodel.html)

## why len is Not a Method

`len(x)` No method is called for the built-in objects of CPython: the length is simply read from a field in a C struct.

In other words, `len` is not called as a method because it gets special treatments as part of the Python data model, just like `abs`. But thanks to the special method `__len__`, you can also make `len` work with your own custom objects.

## Chapter Summary

## Futher Reading

**Magic Methods**

The Ruby community calls their equivalent of the special methods magic methods. Many in Python community adopt. that term as well. I believe the special methods are actually the opposite of magic.

**Metaobjects**

The Art of the Metaobject Protocol(AMOP) is my favorite computer book title. The term metaobject protocol is useful to think about the Python data model and similar features in other language. The metaobejct part refers to the objects that are the building blocks of the language itself. In this context, protocol is a synonym of interface. So a metaobject protocol is a fancy synonym for object model: an API for core language constructs.

## References

* https://docs.python.org/3/reference/datamodel.html